In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)


In [3]:
driver.get("https://www.cricbuzz.com/cricket-series")
time.sleep(3)

In [4]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [5]:
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()


In [6]:
series_data = []
series_blocks = soup.find_all("a", href=True)

for block in series_blocks:
    if "/cricket-series/" in block['href'] and block.text.strip():
        name = block.text.strip()
        url = "https://www.cricbuzz.com" + block['href']
        year = ''.join(filter(str.isdigit, name.split()[-1]))  # get year from end
        series_data.append({
            "Series Name": name,
            "Series Year": year,
            "Series Link": url
        })

In [7]:
df = pd.DataFrame(series_data).drop_duplicates(subset=["Series Link"])
df.to_csv("cricbuzz_series_all_years.csv", index=False)
print("✅ Scraping Complete. Output saved to 'cricbuzz_series_all_years.csv'")

✅ Scraping Complete. Output saved to 'cricbuzz_series_all_years.csv'
